# Using the ImportExcel module to move data
The ImportExcel module is maintained by PowerShell MVP Doug Finke @dfinke on twitter.

You will need to install the module from the PowerShell Gallery, if you haven't done so already.

In [0]:
Install-Module ImportExcel

Before we get started importing data from Excel into our SQL Server, we can use the `Get-ExcelSheetInfo` below to inspeact the multiple worksheets inside an Excel file.

In [2]:
Get-ExcelSheetInfo -Path C:\temp\AW\AdventureWorksDW2017_Dims.xlsx


Name                  Index  Hidden Path                                     
----                  -----  ------ ----                                     
DimScenario               1 Visible C:\temp\AW\AdventureWorksDW2017_Dims.xlsx
DimAccount                2 Visible C:\temp\AW\AdventureWorksDW2017_Dims.xlsx
DimCurrency               3 Visible C:\temp\AW\AdventureWorksDW2017_Dims.xlsx
DimCustomer               4 Visible C:\temp\AW\AdventureWorksDW2017_Dims.xlsx
DimDate                   5 Visible C:\temp\AW\AdventureWorksDW2017_Dims.xlsx
DimDepartmentGroup        6 Visible C:\temp\AW\AdventureWorksDW2017_Dims.xlsx
DimEmployee               7 Visible C:\temp\AW\AdventureWorksDW2017_Dims.xlsx
DimGeography              8 Visible C:\temp\AW\AdventureWorksDW2017_Dims.xlsx
DimOrganization           9 Visible C:\temp\AW\AdventureWorksDW2017_Dims.xlsx
DimProduct               10 Visible C:\temp\AW\AdventureWorksDW2017_Dims.xlsx
DimProductCategory       11 Visible C:\temp\AW\AdventureWorksDW

Use the technique below to import multiple worksheets from an Excel file, and create a new table in SQL Server for each Worksheet.

In [7]:
foreach ($Worksheet in Get-ExcelSheetInfo -Path C:\temp\AW\AdventureWorksDW2017_Dims.xlsx)
{
"$($Worksheet.Path) and $($Worksheet.Name)"
}

C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimScenario
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimAccount
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimCurrency
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimCustomer
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimDate
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimDepartmentGroup
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimEmployee
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimGeography
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimOrganization
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimProduct
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimProductCategory
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimProductSubcategory
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimPromotion
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimReseller
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimSalesReason
C:\temp\AW\AdventureWorksDW2017_Dims.xlsx and DimSalesTerritory


In [8]:
$Worksheet


Name              Index  Hidden Path                                     
----              -----  ------ ----                                     
DimSalesTerritory    16 Visible C:\temp\AW\AdventureWorksDW2017_Dims.xlsx




In [10]:
,(Import-Excel -Path $Worksheet.Path -WorksheetName $Worksheet.Name) |
Write-SqlTableData -ServerInstance localhost\SQL2017 -Database BlankDB -SchemaName Excel -TableName $_.Name -Force;

Import-Excel : A parameter cannot be found that matches parameter name 'WorksheetNamev'.
At line:1 char:42
+ . { ,(Import-Excel -Path $Worksheet.Path -WorksheetNamev $Worksheet.N ...
+                                          ~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidArgument: (:) [Import-Excel], ParameterBindingException
    + FullyQualifiedErrorId : NamedParameterNotFound,Import-Excel
 


In [1]:
foreach ($Worksheet in Get-ExcelSheetInfo -Path C:\temp\AW\AdventureWorksDW2017_Dims.xlsx)
{
,(Import-Excel -Path $Worksheet.Path -WorksheetName $Worksheet.Name) |
Write-SqlTableData -ServerInstance localhost\SQL2017 -Database BlankDB -SchemaName Excel -TableName $Worksheet.Name -Force;
}

Write-SqlTableData : Input string was not in a correct format.Couldn't store <> in ParentEmployeeKey Column.  Expected 
type is Double.
At line:4 char:1
+ Write-SqlTableData -ServerInstance localhost\SQL2017 -Database BlankD ...
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : WriteError: ([Excel].[DimEmployee]:Table) [Write-SqlTableData], ArgumentException
    + FullyQualifiedErrorId : WriteToTableFailure,Microsoft.SqlServer.Management.PowerShell.WriteSqlTableData
 
Write-SqlTableData : Input string was not in a correct format.Couldn't store <T2P 2G8> in PostalCode Column.  Expected 
type is Double.
At line:4 char:1
+ Write-SqlTableData -ServerInstance localhost\SQL2017 -Database BlankD ...
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : WriteError: ([Excel].[DimGeography]:Table) [Write-SqlTableData], ArgumentException
    + FullyQualifiedErrorId : WriteToTableFailure,Microsoft.S

In [14]:
dir SQLSERVER:\SQL\localhost\SQL2017\Databases\BlankDB\Tables


Schema                   Name                        Row Count   Storage      Index  Created               
                                                               Space Used Space Used                       
------                   ----                        --------- ---------- ---------- -------               
dbo                      TransactionHistory             113443    7.15 MB    8.00 KB 4/4/2019 9:33 AM      
Excel                    DimAccount                         99   16.00 KB    8.00 KB 11/5/2019 3:17 PM     
Excel                    DimCurrency                       105    8.00 KB    8.00 KB 11/5/2019 3:17 PM     
Excel                    DimCustomer                     18484    8.48 MB    8.00 KB 11/5/2019 3:17 PM     
Excel                    DimDate                          3652  760.00 KB    8.00 KB 11/5/2019 3:18 PM     
Excel                    DimDepartmentGroup                  7    8.00 KB    8.00 KB 11/5/2019 3:18 PM     
Excel                    Di

In [0]:
dir C:\temp\AW -Filter Fact*.xlsx;
#,(Import-Excel -Path C:\temp\Excel\SysColumns_AdventureWorks2014.xlsx) |
#Write-SqlTableData -ServerInstance localhost\SQL2017 -Database BlankDB -SchemaName dbo -TableName MyOtherNewTable_fromExcel -Force